In [5]:
import torch
from llm_from_scratch.c4 import GPTModel, GPTConfig

#1 We shorten the context length from 1,024 to 256 tokens.
#2 It’s possible and common to set dropout to 0.
GPT_CONFIG_124M: GPTConfig = {
    "vocab_size": 50257,
    "context_length": 256,  # 1
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,  # 2
    "qkv_bias": False,
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()
None

In [10]:
import tiktoken
from llm_from_scratch.c4 import generate_text_simple
from llm_from_scratch.c5 import text_to_token_ids, token_ids_to_text

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

ImportError: cannot import name 'generate_text_simple' from 'llm_from_scratch.c4' (/private/var/tmp/_bazel_hoshyari/e7198472d5e413c18f06cfb8f623b4c8/execroot/_main/bazel-out/darwin_arm64-dbg/bin/llm_from_scratch/c5/notebook.runfiles/_main/llm_from_scratch/c4/__init__.py)